# **Introduction to CUDA and PyCUDA**

PyCUDA gives you easy, Pythonic access to Nvidia’s CUDA parallel computation API. 

*   Abstractions make CUDA programming easier
*   Full power of CUDA API if needed
*   Automatic error checking and cleanup


## **Initialization**
A few modules have to be loaded to initialize communication to the GPU:

*   import pycuda.driver as cuda
*   import pycuda.autoinit

The pycuda.driver module has methods that:
1. Allocate memory on the GPU (cuda.mem alloc())
2. Copy numpy arrays to the GPU (cuda.memcpy htod())
3. Execute kernels on the GPU described by CUDA code (see compiler.SourceModule)
4. Copy data from the GPU back to numpy arrays (cuda.memcpy dtoh()).

In [1]:
# Import PyCUDA and several modules associated with the PyCUDA
!pip install pycuda 
import pycuda
import pycuda.driver as cuda
cuda.init()

import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
from pycuda.curandom import rand as curand
from pycuda.elementwise import ElementwiseKernel

import numpy as np
import numpy.linalg as la

/bin/sh: 1: pip: not found


## **CUDA device query**

A GPU query is a very basic operation that will tell us the specific technical details of our GPU, such as available GPU memory and core count.

In [2]:
print('CUDA device query (PyCUDA version) \n')
print('Detected {} CUDA Capable device(s) \n'.format(cuda.Device.count()))
for i in range(cuda.Device.count()):
    
    gpu_device = cuda.Device(i)
    print('Device {}: {}'.format( i, gpu_device.name() ) )
    compute_capability = float( '%d.%d' % gpu_device.compute_capability() )
    print('\t Compute Capability: {}'.format(compute_capability))
    print('\t Total Memory: {} megabytes'.format(gpu_device.total_memory()//(1024**2)))

CUDA device query (PyCUDA version) 

Detected 1 CUDA Capable device(s) 

Device 0: NVIDIA Tegra X1
	 Compute Capability: 5.3
	 Total Memory: 3962 megabytes


## **Basics of GPU programming**

We'll see how to write and read data to and from the GPU's memory, and how to write some very simple elementwise GPU functions in CUDA C.

### PyCUDA's gpuarray class

PyCUDA's gpuarray class has important role within GPU programming in Python. This has all of the features from NumPy—multidimensional vector/matrix/tensor shape structuring, array-slicing, array unraveling, and overloaded operators for point-wise computations (for example, +, -, *, /, and **).

### Transfering data to and from the GPU with gpuarray

GPU has its own memory apart from the host computer's memory, which is known as device memory. 
In CUDA C, data is transfferd back and forth between the CPU to the GPU (with commands such as cudaMemcpyHostToDevice and cudaMemcpyDeviceToHost).

Fortunately, PyCUDA covers all of the overhead of memory allocation, deallocation, and data transfers with the gpuarray class.

In [3]:
#contain the host data
host_data = np.array([1,2,3,4,5], dtype=np.float32)
#transfer the host data to the GPU and create a new GPU array
device_data = gpuarray.to_gpu(host_data)
#perform pointwise multiplication on the GPU
device_data_x2 = 2* device_data
#retrieve the GPU data into a new with the gpuarray.get function
host_data_x2 = device_data_x2.get()
print(host_data_x2)

[ 2.  4.  6.  8. 10.]


## **Example: Doubling the value of elements in an array**

Here, we will take an array and double the element of it on the GPU.

### Step1: Getting started

In [4]:
# Declare the array as follows:
np.random.seed(1729)
a = np.random.randn(4,4).astype(np.float32)

### Step 2: Transferring Data to the GPU

The next step in most programs is to transfer data onto the device. In PyCuda, you will mostly transfer data from numpy arrays on the host.

In [5]:
# firstly, we need to allocate memory on the device
a_gpu = cuda.mem_alloc(a.nbytes)

In [6]:
# we need to transfer the data to the GPU
cuda.memcpy_htod(a_gpu, a)

### Step 3: Executing a Kernel

For this tutorial, we will write code to double each entry in a_gpu. To this end, we write the corresponding CUDA C code, and feed it into the constructor of a [pycuda.compiler.SourceModule](https://documen.tician.de/pycuda/driver.html#pycuda.compiler.SourceModule):

In [7]:
# Double each entry in the variable a_gpu
mod = SourceModule("""
  __global__ void twice(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

If there aren’t any errors, the code is now compiled and loaded onto the device. We find a reference to [pycuda.driver.Function](https://documen.tician.de/pycuda/driver.html#pycuda.driver.Function) and call it, specifying a_gpu as the argument, and a block size of 4x4:

In [8]:
func = mod.get_function("twice")
func(a_gpu, block=(4,4,1))

Finally, we fetch the data back from the GPU and display it, together with the original a:

In [9]:
a_doubled = np.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)

print("a",a)
print("\nDoubled value",a_doubled)

a [[-0.6873394  -0.82099473  1.6523609  -0.57529306]
 [ 1.0989678   0.92594606 -0.9934138  -0.8582211 ]
 [ 0.07488676  0.5293555   0.12095155 -0.22442362]
 [-1.5566785   0.05594088  0.16147153 -2.1346416 ]]

Doubled value [[-1.3746789  -1.6419895   3.3047218  -1.1505861 ]
 [ 2.1979356   1.8518921  -1.9868276  -1.7164422 ]
 [ 0.14977352  1.058711    0.2419031  -0.44884723]
 [-3.113357    0.11188176  0.32294306 -4.2692833 ]]


### Abstracting Away the Complications

Using a [pycuda.gpuarray.GPUArray](https://documen.tician.de/pycuda/array.html#pycuda.gpuarray.GPUArray), the same effect can be achieved with much less writing:

In [10]:
# implementing with gpu_array
a_gpu = gpuarray.to_gpu(a)
a_doubled = (2*a_gpu).get()
print("a",a)
print("\nDoubled value",a_doubled)

a [[-0.6873394  -0.82099473  1.6523609  -0.57529306]
 [ 1.0989678   0.92594606 -0.9934138  -0.8582211 ]
 [ 0.07488676  0.5293555   0.12095155 -0.22442362]
 [-1.5566785   0.05594088  0.16147153 -2.1346416 ]]

Doubled value [[-1.3746789  -1.6419895   3.3047218  -1.1505861 ]
 [ 2.1979356   1.8518921  -1.9868276  -1.7164422 ]
 [ 0.14977352  1.058711    0.2419031  -0.44884723]
 [-3.113357    0.11188176  0.32294306 -4.2692833 ]]


### Shortcuts for Explicit Memory Copies

The [pycuda.driver.In](https://documen.tician.de/pycuda/driver.html#pycuda.driver.In), [pycuda.driver.Out](https://documen.tician.de/pycuda/driver.html#pycuda.driver.Out), and [pycuda.driver.InOut](https://documen.tician.de/pycuda/driver.html#pycuda.driver.InOut) argument handlers can simplify some of the memory transfers. For example, instead of creating a_gpu, if replacing a is fine, the following code can be used:

In [11]:
# implementing with InOut
func(cuda.InOut(a), block=(4, 4, 1))
print("a",a)

a [[-1.3746789  -1.6419895   3.3047218  -1.1505861 ]
 [ 2.1979356   1.8518921  -1.9868276  -1.7164422 ]
 [ 0.14977352  1.058711    0.2419031  -0.44884723]
 [-3.113357    0.11188176  0.32294306 -4.2692833 ]]


## **Example: Addition of two 1D arrays**

Here, we will add two 1D arrays and execute it on GPU.

In [12]:
# declare arrays
m = np.random.randn(10).astype(np.float32)
n = np.random.randn(10).astype(np.float32)

# execute the kernel
mod2_1D = SourceModule("""
__global__ void sum2arr(float *sum, float *m, float *n)
{
  const int i = threadIdx.x;
  sum[i] = m[i] + n[i];
}
""")

func = mod2_1D.get_function("sum2arr")

# handle memory transfer with 'Out()'
sum_1D = np.zeros_like(m)
func(cuda.Out(sum_1D),
     cuda.In(m), cuda.In(n),
     block=(10,1,1))

# print result
print("m",m)
print("\nn",n)
print("\nsum",sum_1D)

m [ 0.10967004  0.44301215  0.39626622  0.2497974   1.2984973  -1.2804337
 -0.97546583 -0.26908663 -1.1057384  -0.1279927 ]

n [-0.61782736 -0.98912627 -2.8598924  -0.7943475  -0.30579695  1.7006376
 -0.63544416  0.00457387  1.133304    0.14164127]

sum [-0.5081573  -0.5461141  -2.4636261  -0.5445501   0.99270034  0.42020392
 -1.6109099  -0.26451278  0.0275656   0.01364857]


## **Example: Addition of matrices**

Here, we will add two matrices and execute it on GPU.

In [13]:
# declare matrices
b = np.random.randn(4,4).astype(np.float32)
c = np.random.randn(4,4).astype(np.float32)

# execute the kernel
mod2_2D = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

func = mod2_2D.get_function("add2")

# transfer the data to the GPU
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)

func(b_gpu,c_gpu, block=(4,4,1))

sum_2D = np.empty_like(b)
cuda.memcpy_dtoh(sum_2D, b_gpu)

# print result
print("b",b)
print("\nc",c)
print("\nsum",sum_2D)

b [[ 2.091718   -1.2695593  -0.7228234  -0.01938889]
 [-0.18282357 -1.1190658  -0.6982751   2.1070845 ]
 [-0.0025556   0.40901005  0.28363675 -1.9498545 ]
 [ 0.38603088  0.23064111 -1.1869708  -0.6665516 ]]

c [[ 1.7977669   0.66379833  1.1238116  -1.1343062 ]
 [ 1.1844498   0.70126235  0.45549637 -0.0814869 ]
 [-1.0958686   0.4407169   1.6427085   0.4479594 ]
 [-0.7867167  -0.17314298  2.319482   -0.32860455]]

sum [[ 3.889485   -0.60576093  0.40098822 -1.1536951 ]
 [ 1.0016263  -0.4178034  -0.24277872  2.0255976 ]
 [-1.0984242   0.8497269   1.9263453  -1.5018951 ]
 [-0.40068582  0.05749813  1.1325113  -0.99515617]]


##**Example: Multiplication of matrices**

Here, we will multiple two matrices and execute it on GPU.

In [ ]:
# declare matrices
r = np.random.randn(10).astype(np.float32)
s = np.random.randn(10).astype(np.float32)

# execute kernel
mod3 = SourceModule("""
__global__ void multiply2arr(float *dest, float *r, float *s)
{
  const int i = threadIdx.x;
  dest[i] = r[i] * s[i];
}
""")

func = mod3.get_function("multiply2arr")

product = np.zeros_like(r)

# handle memory transfer
func(cuda.Out(product),
     cuda.In(r), cuda.In(s),
     block=(10,1,1))

# print result
print("r",r)
print("\ns",s)
print("\nproduct",product)

## **Example: Linear combination of variables**

The functionality in the module [pycuda.elementwise](https://documen.tician.de/pycuda/array.html#module-pycuda.elementwise) contains tools to help generate kernels that evaluate multi-stage expressions on one or several operands in a single pass. Here's a usage example:

In [17]:
# generate arrays of random numbers using curand()
n1_gpu = curand((10,))
n2_gpu = curand((10,))

# generate a kernel that takes a number of scalar or vector arguments and performs the scalar operation on each entry of its arguments, if that argument is a vector.
linear_combination = ElementwiseKernel(
        "float a, float *x, float b, float *y, float *z",
        "z[i] = my_f(a*x[i], b*y[i])",
        "linear_combination",
        preamble="""
        __device__ float my_f(float x, float y)
        { 
          return sin(x*y);
        }
        """)

# make a new, uninitialized GPUArray having the same properties as other_ary
c_gpu = gpuarray.empty_like(n1_gpu)

# call the function
linear_combination(5, n1_gpu, 6, n2_gpu, c_gpu)

# test for a particular condition
assert la.norm(c_gpu.get() - np.sin((5*n1_gpu*6*n2_gpu).get())) < 1e-5

## **Example: Numba and PyCUDA**

The module [pycuda.autoprimaryctx](https://documen.tician.de/pycuda/util.html#module-pycuda.autoprimaryctx) is similar to [pycuda.autoinit](https://documen.tician.de/pycuda/util.html#module-pycuda.autoinit), except that it retains the device primary context instead of creating a new context in [pycuda.tools.make_default_context()](https://documen.tician.de/pycuda/util.html#pycuda.tools.make_default_context). Notably, it also has device and context attributes.

In [16]:
from numba import cuda

# Using autoprimaryctx instead of autoinit since Numba can only operate on a primary context
import pycuda.autoprimaryctx  

# Creating a PyCUDA gpuarray
print("a",a_gpu)

# Numba kernel
@cuda.jit
def double(x):
    i, j = cuda.grid(2)

    if i < x.shape[0] and j < x.shape[1]:
        x[i, j] *= 2

# Calling the Numba kernel on the PyCUDA gpuarray, using the CUDA Array Interface transparently
double[(4, 4), (1, 1)](a_gpu)
print("Doubling values using numba: ",a_gpu)

a [[-0.6873394  -0.82099473  1.6523609  -0.57529306]
 [ 1.0989678   0.92594606 -0.9934138  -0.8582211 ]
 [ 0.07488676  0.5293555   0.12095155 -0.22442362]
 [-1.5566785   0.05594088  0.16147153 -2.1346416 ]]
Doubling values using numba:  [[-1.3746789  -1.6419895   3.3047218  -1.1505861 ]
 [ 2.1979356   1.8518921  -1.9868276  -1.7164422 ]
 [ 0.14977352  1.058711    0.2419031  -0.44884723]
 [-3.113357    0.11188176  0.32294306 -4.2692833 ]]


# **Exercise**

- Write a cuda kernel to find the elementwise square of a matrix
- Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero
- Write a cuda kernel to multiply two matrices:
  - Assume square matrices, with dimensions < 1024
  - Assume square matrices, with dimensions > 1024
  - Assume non-square matrices, with dimensions > 1024

In [19]:
import numpy as np
from numba import cuda

@cuda.jit
def square_matrix(A):
    row, col = cuda.grid(2)
    N = A.shape[0]  # Assuming square matrix

    if row < N and col < N:
        A[row, col] *= A[row, col]  # Square each element

# Define matrix size
N = 4  # Example size
A = np.random.rand(N, N).astype(np.float32)  # Random matrix

# Copy to device
d_A = cuda.to_device(A)

# Define thread and block configuration
threads_per_block = (16, 16)
blocks_per_grid = ((N + threads_per_block[0] - 1) // threads_per_block[0],
                   (N + threads_per_block[1] - 1) // threads_per_block[1])

# Launch kernel
square_matrix[blocks_per_grid, threads_per_block](d_A)

# Copy result back to host
A_squared = d_A.copy_to_host()
print(A_squared)


[[0.01937819 0.42511994 0.33997074 0.63602585]
 [0.03652494 0.4406518  0.8636695  0.76425326]
 [0.03750517 0.09043005 0.06292532 0.02009321]
 [0.25513023 0.20018847 0.63525164 0.25055218]]


In [22]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Define CUDA kernel code as a string
kernel_code = """
__global__ void negateMatrix(float *A, float *B, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        int index = row * N + col;
        B[index] = -A[index];  // Compute negative
    }
}
"""

# Define matrix size
N = 4  # Example size
A = np.random.rand(N, N).astype(np.float32)  # Random matrix
B = np.zeros_like(A)  # Matrix to store the negated result

# Allocate memory on the device
A_gpu = cuda.mem_alloc(A.nbytes)
B_gpu = cuda.mem_alloc(B.nbytes)

# Copy data to device
cuda.memcpy_htod(A_gpu, A)

# Compile the kernel
mod = SourceModule(kernel_code)

# Get the kernel function
negate_matrix = mod.get_function("negateMatrix")

# Define thread and block configuration
threads_per_block = (16, 16, 1)
blocks_per_grid = ((N + threads_per_block[0] - 1) // threads_per_block[0],
                   (N + threads_per_block[1] - 1) // threads_per_block[1])

# Launch the kernel
negate_matrix(A_gpu, B_gpu, np.int32(N), block=threads_per_block, grid=blocks_per_grid)

# Copy result back to host
cuda.memcpy_dtoh(B, B_gpu)

print("Original Matrix A:\n", A)
print("Negated Matrix B:\n", B)


Original Matrix A:
 [[0.2995714  0.99545646 0.7228736  0.3163466 ]
 [0.52194184 0.71298903 0.54312956 0.6573724 ]
 [0.05103904 0.9879216  0.530643   0.27212656]
 [0.16180052 0.9762489  0.7255236  0.04226469]]
Negated Matrix B:
 [[-0.2995714  -0.99545646 -0.7228736  -0.3163466 ]
 [-0.52194184 -0.71298903 -0.54312956 -0.6573724 ]
 [-0.05103904 -0.9879216  -0.530643   -0.27212656]
 [-0.16180052 -0.9762489  -0.7255236  -0.04226469]]


In [24]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Define CUDA kernel code for matrix multiplication
kernel_code = """
__global__ void matrix_multiply(float *A, float *B, float *C, int N, int M, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < K) {
        float value = 0.0;
        for (int i = 0; i < M; i++) {
            value += A[row * M + i] * B[i * K + col];  // Matrix multiplication
        }
        C[row * K + col] = value;  // Store result in matrix C
    }
}
"""

# Matrix dimensions
N = 1024  # Row size of A and C (and square matrices for case 1)
M = 1024  # Column size of A and row size of B (and square matrices for case 2)
K = 1024  # Column size of B (and square matrices for case 3)

# For square matrices with dimensions < 1024, N, M, K < 1024
# A = np.random.rand(N, M).astype(np.float32)
# B = np.random.rand(M, K).astype(np.float32)
# C = np.zeros((N, K), dtype=np.float32)

# For non-square matrices with dimensions > 1024, using larger N, M, K
N_large = 2048  # Example for non-square matrix A
M_large = 4096  # Example for non-square matrix B
K_large = 2048  # Resulting matrix C

A_large = np.random.rand(N_large, M_large).astype(np.float32)
B_large = np.random.rand(M_large, K_large).astype(np.float32)
C_large = np.zeros((N_large, K_large), dtype=np.float32)

# Allocate memory on the device
A_gpu = cuda.mem_alloc(A_large.nbytes)
B_gpu = cuda.mem_alloc(B_large.nbytes)
C_gpu = cuda.mem_alloc(C_large.nbytes)

# Copy matrices to device
cuda.memcpy_htod(A_gpu, A_large)
cuda.memcpy_htod(B_gpu, B_large)

# Compile the kernel
mod = SourceModule(kernel_code)

# Get the kernel function
matrix_multiply = mod.get_function("matrix_multiply")

# Define thread and block configuration
threads_per_block = (16, 16, 1)
blocks_per_grid = ((K_large + threads_per_block[0] - 1) // threads_per_block[0],
                   (N_large + threads_per_block[1] - 1) // threads_per_block[1])

# Launch the kernel for matrix multiplication
matrix_multiply(A_gpu, B_gpu, C_gpu, np.int32(N_large), np.int32(M_large), np.int32(K_large),
                 block=threads_per_block, grid=blocks_per_grid)

# Copy result back to host
cuda.memcpy_dtoh(C_large, C_gpu)

print("Resulting Matrix C:\n", C_large)


Resulting Matrix C:
 [[1020.9612  1013.0582  1019.6845  ... 1017.9617  1027.5701   995.62524]
 [1021.8103  1011.1824  1013.0184  ... 1022.8366  1022.0986  1004.51575]
 [1021.0074  1004.4751  1007.10974 ... 1017.53357 1024.4464  1006.3561 ]
 ...
 [1040.8395  1021.2631  1030.9387  ... 1036.1636  1038.9098  1024.4077 ]
 [1028.2521  1014.87695 1029.2435  ... 1037.9235  1023.98456 1019.856  ]
 [1052.5536  1041.3102  1046.9324  ... 1058.4493  1065.2323  1045.659  ]]
